In [2]:
import streamlit as st


In [3]:
import streamlit as st
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
import os

In [9]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv("OPENAI_API_KEY")

In [10]:
# Cache the function to load and process PDF documents
@st.cache(allow_output_mutation=True)
def load_and_process_pdfs(pdf_folder_path):
    documents = []
    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, file)
            loader = PyPDFLoader(pdf_path)
            documents.extend(loader.load())
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)
    return splits

2024-11-10 17:14:36.238 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 17:14:36.239 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 17:14:36.240 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new
caching commands, `st.cache_data` or `st.cache_resource`. More information
[in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching
logic used by `st.cache_data` and `st.cache_resource`. This might lead to some problems
or unexpected behavior in certain edge cases.



In [14]:
# Import cache_resource from Streamlit
from streamlit import cache_resource
@st.cache(allow_output_mutation=True)
# Cache the function to initialize the vector store with documents
#@cache_resource
def initialize_vectorstore(splits):
    embeddings = OpenAIEmbeddings(api_key=st.secrets["openai"]["api_key"])
    return FAISS.from_documents(documents=splits, embedding=embeddings)

pdf_folder_path = "./fin_ed_docs"
splits = load_and_process_pdfs(pdf_folder_path)
vectorstore = initialize_vectorstore(splits)


2024-11-10 17:22:05.547 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 17:22:05.548 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 17:22:05.549 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new
caching commands, `st.cache_data` or `st.cache_resource`. More information
[in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching
logic used by `st.cache_data` and `st.cache_resource`. This might lead to some problems
or unexpected behavior in certain edge cases.

2024-11-10 17:22:05.551 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-10 17:22:05.551 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-1

UnhashableParamError: Cannot hash argument 'splits' (of type `builtins.list`) in 'initialize_vectorstore'.

To address this, you can tell Streamlit not to hash this argument by adding a
leading underscore to the argument's name in the function signature:

```
@st.cache_resource
def initialize_vectorstore(_splits, ...):
    ...
```
            

In [ ]:
prompt_template = """You are a finance expert. You need to answer the question related to finance. 
Given below is the context and question of the user. Don't answer question outside the context provided.
context = {context}
question = {question}
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key="Your Key")
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
rag_chain = (
    {"context": vectorstore.as_retriever() | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)